# GIN 実験

In [2]:
import sys
sys.path.append("../codes")

In [3]:
from GIN import GIN

from set_data_folder import make_train_data
from optuna_cv_utils import cv_train, cv_test, make_datasets

In [5]:
m_list = [2, 4, 6, 8, 10]
p_list = {"100": [0.04, 0.08, 0.11, 0.15, 0.19], "1000":[0.004, 0.008, 0.012, 0.016, 0.02]}
step_list = {"100": [200, 384, 564, 736, 900], "1000":[1996, 3984, 5964, 7936, 9900]}

cnt = 0
for node in ["100", "1000"]:
    for m, p, step in zip(m_list, p_list[node], step_list[node]):
        # make train data folder
        p_s = [
            {"kind": "barabasi", "node": [node], "p": [str(m)]},
            {"kind": "noGrowth", "node": [node], "p": [str(step)]},
            {"kind": "noAttach", "node": [node], "p": [str(m)]},
            {"kind": "random", "node": [node], "p": [str(p)]}
        ]

        # train data folderの作成
        make_train_data(p_s, "../train_data/net").copy_data()
        # dataset の作成
        data_list = make_datasets()
        break
    break

100%|██████████| 5000/5000 [00:09<00:00, 505.45it/s]


In [7]:
model = GIN(1, 4, config={"aggregation": "sum", "dropout": .5, "hidden_units": [32, 32, 32, 32],"train_eps": True})

In [12]:
from torch_geometric.data import DataLoader
from sklearn.model_selection import KFold
from torch.utils.data.dataset import Subset

In [13]:
# 交差検証
fold = KFold(
    n_splits=10, shuffle=True, random_state=0
)

valid_accs = []
for fold_idx, (train_idx, valid_idx) in enumerate(fold.split(data_list)):
    # split data
    train_loader = DataLoader(
        Subset(data_list, train_idx),
        shuffle=True,
        batch_size=32,
    )
    valid_loader = DataLoader(
        Subset(data_list, valid_idx),
        shuffle=False,
        batch_size=32,
    )


In [16]:
 for data in train_loader:
        data

In [17]:
data

Batch(batch=[1600], edge_index=[2, 6332], ptr=[17], x=[1600, 1], y=[16])

In [18]:
model(data)

tensor([[   2.5753,  -22.8295,  -78.5072,   51.5859],
        [  63.0097,  -24.0562,  -71.6452,   45.5452],
        [   0.0000,   21.0925,   46.2954,   50.2420],
        [ -72.5849,   61.1649, -111.4313,   -8.4696],
        [  78.8011,  -57.3528,  -37.2146,    3.1903],
        [  21.2603,  -37.2949,   15.4775,  -24.4933],
        [   0.0000,  -60.1459,   85.9983,    9.2062],
        [  54.8880,   -7.4800,  -40.4171,   49.8804],
        [ -77.4148,  -87.4484, -114.1102,    8.1898],
        [ -60.2667,  209.0860, -246.1641,  125.7522],
        [  11.5964,   74.4288,  203.7564,  159.2501],
        [  11.2604,  -57.8407,   35.1956,   64.5693],
        [ -18.3615,   54.4935,  -74.7644,   -8.9300],
        [ -16.6463,   -1.5691,  -46.5515,  -39.8660],
        [  20.8994,   25.2620,  -40.9158,  -23.6744],
        [ -10.5884,  -50.5821,  -73.7469,    0.0000]], grad_fn=<AddBackward0>)